In [4]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
from nltk import word_tokenize, ngrams
import argparse
import os
import re
import time

from nltk import sent_tokenize
import evaluate


In [7]:
def get_all_filepaths(dir):
    file_paths = {}
    titles = ["head_0", "head_1", "mixed"]
    for title in titles:
        path = os.path.join(dir, title)
        path = os.path.join(path,"test_outfinal.txt")
        file_paths[title] = path
    return file_paths
def get_paired_outputs(filepath):
    paired_outputs = {}
    with open(filepath, "r") as f:
        lines = f.readlines()
        cur_index = 0
        for index , line in enumerate(lines):
            if index % 4 == 0:
                paired_outputs[cur_index] = {"article": line}
            elif index % 4 == 1:
                paired_outputs[cur_index]["reference"] = line
            elif index % 4 == 2:
                paired_outputs[cur_index]["summary"] = line
            elif index % 4 == 3:
                cur_index += 1
    return paired_outputs
def get_rouge_scores(references, candidates):
    rouge = evaluate.load("rouge")
    results = rouge.compute(predictions=candidates, references=references)
    print(results)




def read_file(input_dict):

    inputs = []
    candidates = []
    references = []

    for key in input_dict.keys():
        input = input_dict[key]['article']
        gold = input_dict[key]['reference']
        summary = input_dict[key]['summary']
        input = input.strip()
        input = re.sub('<.*?>', '', input)
        gold = gold.strip()
        gold = re.sub('<.*?>', '', gold)
        summary = summary.strip()
        summary = re.sub('<.*?>', '', summary)
        inputs.append(input)
        candidates.append(summary)
        references.append(gold)

    return inputs, references, candidates



def compute_rouge(input_dict):
    inputs, references, candidates = read_file(input_dict)
    get_rouge_scores(references, candidates)

def get_overlap(inp, out, ngram):
    grams_inp = set(ngrams(word_tokenize(inp.lower()), ngram))
    grams_out = set(ngrams(word_tokenize(out.lower()), ngram))

    total = len(grams_out)
    common = len(grams_inp.intersection(grams_out))
    if total == 0:
        return 0
    else:
        return float(common) / float(total)
def get_overlap_file(input_dict , output_filename, ngram=2, graph=False):
    """ draws and stores the ngram overlap of the generated and gold summaries with the input 
        ARGS : 
            input_dict : input dict where key is index, where each output is of the form article , reference, summary
            output_filename : used to store the resulting plot
            ngram : the size of the ngram overlap to be considered
            graph : whether to draw the graph or not

    
    """

    overlap_gold = []
    overlap_gen = []
    gen_length = []
    gold_length = []

    for key in input_dict.keys():
        inp = input_dict[key]['article']
        gold = input_dict[key]['reference']
        out = input_dict[key]['summary']


        overlap_gold.append(get_overlap(inp, gold, ngram))
        overlap_gen.append(get_overlap(inp, out, ngram))

        gen_length.append(len(out.split(' ')))
        gold_length.append(len(gold.split(' ')))


    overlap_gold_mean = np.mean(overlap_gold)
    overlap_gen_mean = np.mean(overlap_gen)
    gen_length = np.mean(gen_length)
    gold_length = np.mean(gold_length)


    #print(f'Gold overlap %dgram = %f' % (ngram, overlap_gold_mean))
    print(f'Generated overlap %dgram = %f' % (ngram, overlap_gen_mean))

    #print(f'Gold length = %f' % gold_length)
    print(f'Generated length = %f' % gen_length)

    if graph:
        # the histogram of the data
        kwargs = dict(histtype='stepfilled', alpha=0.5, density=True, bins=80)

        weights = np.ones_like(overlap_gold) / float(len(overlap_gold))
        plt.hist(overlap_gold, **kwargs, label='gold', weights=weights)

        weights = np.ones_like(overlap_gen) / float(len(overlap_gold))
        plt.hist(overlap_gen, **kwargs, label='generated', weights=weights)



        plt.xlabel(f'{ngram}-gram overlap')
        plt.ylim(0, 8)
        #plt.xlim(0, 1)
        plt.legend()
        plt.grid(True)
        plt.savefig(output_filename)
        plt.clf()
    return overlap_gold, overlap_gen
def get_all_experiment_filepaths(exp_name = 'exp_0'):
    experiment_path = os.path.join("./outputs", exp_name)
    all_dir_path = [os.path.join(experiment_path, dir) for dir in os.listdir(experiment_path) if os.path.isdir(os.path.join(experiment_path, dir))]
    filepaths = {}
    for dir in all_dir_path:
        basename = os.path.basename(dir)
        filepaths[basename] = os.path.join(dir, "test_outfinal.txt")
    return filepaths
def generate_results(exp_name = "exp_0"):
    # print the experiment 
    print("experiment name : " + str(exp_name))
    #head_0_outputs = get_paired_outputs("./outputs/" + str(exp_name) + "/head_0/test_outfinal.txt")
    #head_1_outputs = get_paired_outputs("./outputs/" + str(exp_name) + "/head_1/test_outfinal.txt")
    #mixed_outputs = get_paired_outputs("./outputs/" + str(exp_name) + "/mixed/test_outfinal.txt")
    #filepaths = get_all_filepaths("outputs/exp_3")
    #head_0_outputs = get_paired_outputs(filepaths["head_0"])
    #head_1_outputs = get_paired_outputs(filepaths["head_1"])
    #mixed_outputs = get_paired_outputs(filepaths["mixed"])
    filepaths = get_all_experiment_filepaths(exp_name)

    print("gold statistic")
    print("gold overlap : {0} and gold length : {1}".format(0.44, 14))

    for mixture_type, filepath in filepaths.items():
        print("for mixture type : " + str(mixture_type))
        outputs = get_paired_outputs(filepath)
        compute_rouge(outputs)
        res = get_overlap_file(outputs, "./outputs/" + str(exp_name) + "/" + str(mixture_type) + "_overlap.jpg", ngram = 2, graph = True)
        print("")
        print("---------------------------------- Done ----------------------------------")


    # print("for head 0 ")
    # compute_rouge(head_0_outputs)
    # head_0_overlap_gold, head_0_overlap_gen = get_overlap_file(head_0_outputs, "./outputs/" + str(exp_name) + "/head_0_overlap.jpg", ngram = 2, graph = True)
    # print("")
    
    # print("for head 1")
    # compute_rouge(head_1_outputs)
    # head_1_overlap_gold, head_1_overlap_gen = get_overlap_file(head_1_outputs, "./outputs/" + str(exp_name) + "/head_1_overlap.jpg", ngram = 2, graph = True)
    # print("")

    # print("for mixed")
    # compute_rouge(mixed_outputs)
    # mixed_overlap_gold, mixed_overlap_gen = get_overlap_file(mixed_outputs, "./outputs/" + str(exp_name) + "/mixed_overlap.jpg", ngram = 2, graph = True)
    # print("")

    print("---------------------------------- All Done ----------------------------------")
    print("")
    print("")

    #print("head 0")
    #head_0_overlap_gold, head_0_overlap_gen = get_overlap_file(head_0_outputs, "./outputs/" + str(exp_name) + "/head_0_overlap.jpg", ngram = 2, graph = True)
    #print("head 1")
    #head_1_overlap_gold, head_1_overlap_gen = get_overlap_file(head_1_outputs, "./outputs/" + str(exp_name) + "/head_1_overlap.jpg", ngram = 2, graph = True)
    # print("mixed")
    # mixed_overlap_gold, mixed_overlap_gen = get_overlap_file(mixed_outputs, "./outputs/" + str(exp_name) + "/mixed_overlap.jpg", ngram = 2, graph = True)
    
    # kwargs = dict(histtype='stepfilled', alpha=0.5, density=True, bins=80)

    # weights = np.ones_like(head_0_overlap_gen) / float(len(head_0_overlap_gold))
    # plt.hist(head_0_overlap_gen, **kwargs, label='head_0', weights=weights)

    # weights = np.ones_like(head_1_overlap_gen) / float(len(head_1_overlap_gold))
    # plt.hist(head_1_overlap_gen, **kwargs, label='head_1', weights=weights)

    # weights = np.ones_like(mixed_overlap_gen) / float(len(mixed_overlap_gold))
    # plt.hist(mixed_overlap_gen, **kwargs, label='mixed', weights=weights)



    # plt.xlabel(f'{2}-gram overlap')
    # plt.ylim(0, 8)
    # #plt.xlim(0, 1)
    # plt.legend()
    # plt.grid(True)
    # plt.savefig("./outputs/" + str(exp_name) + "/combined_overlap.jpg")
    # plt.clf()
    # print("----------------------------- All done ----------------------------------")
    #return head_0_overlap_gold, head_0_overlap_gen, head_1_overlap_gold, head_1_overlap_gen, mixed_overlap_gold, mixed_overlap_gen
    #return head_0_overlap_gold, head_0_overlap_gen, head_1_overlap_gold, head_1_overlap_gen
def do_experiments(list_of_experiments):
    for exp in list_of_experiments:
        results = generate_results(exp)
        print("---------------------------------- Experiment Done ----------------------------------")







In [8]:
#experiment_list = ["baseline_div_loss_0", "baseline_div_loss_0.1","baseline_div_loss_0.5", "baseline_div_loss_1"]
experiment_list = [
    "overlap_supervision_div_loss_0_epoch_1",
    "overlap_supervision_div_loss_0_epoch_2",

    "overlap_supervision_div_loss_0.1_epoch_1",
        "overlap_supervision_div_loss_0.1_epoch_2",

    "overlap_supervision_div_loss_0.5_epoch_1",
    "overlap_supervision_div_loss_0.5_epoch_2",


    "overlap_supervision_div_loss_1_epoch_1",
    "overlap_supervision_div_loss_1_epoch_2",

]
    
do_experiments(experiment_list)

experiment name : overlap_supervision_div_loss_0_epoch_1
gold statistic
gold overlap : 0.44 and gold length : 14
for mixture type : prob_0.75


INFO:absl:Using default tokenizer.


{'rouge1': 0.28696378385877874, 'rouge2': 0.1290580159926566, 'rougeL': 0.24873941571460334, 'rougeLsum': 0.24869075427257956}
Generated overlap 2gram = 0.703914
Generated length = 22.591803

---------------------------------- Done ----------------------------------
for mixture type : prob_0.50


INFO:absl:Using default tokenizer.


{'rouge1': 0.274799603324163, 'rouge2': 0.12974137989851775, 'rougeL': 0.2423226131661041, 'rougeLsum': 0.24187799104409674}
Generated overlap 2gram = 0.840950
Generated length = 23.379508

---------------------------------- Done ----------------------------------
for mixture type : head_0


...

INFO:absl:Using default tokenizer.


{'rouge1': 0.25013684646228096, 'rouge2': 0.11273797418289688, 'rougeL': 0.2198814019147538, 'rougeLsum': 0.21979617336075782}
Generated overlap 2gram = 0.880611
Generated length = 23.099180

---------------------------------- Done ----------------------------------
for mixture type : head_1


INFO:absl:Using default tokenizer.


{'rouge1': 0.24077199656950682, 'rouge2': 0.1072698477490533, 'rougeL': 0.21280574300713706, 'rougeLsum': 0.21253481665566434}
Generated overlap 2gram = 0.889769
Generated length = 23.160656

---------------------------------- Done ----------------------------------
---------------------------------- All Done ----------------------------------


---------------------------------- Experiment Done ----------------------------------


<Figure size 432x288 with 0 Axes>